# Uncertainties and Fitting

In [ ]:
import pandas
import math
import matplotlib.pyplot as plt
from random import seed
from random import random
from random import gauss
import numpy as np
from scipy.optimize import curve_fit

**Counting Uncertainties**

Here we're going to generate *ndatasets* of size *ntries* of random numbers distributed according to a Gaussian with $\mu = 0$ and $\sigma = 1$.  The distinction between *ndatasets* and *ntries* is artifical; we're really just generating *ndatasets x ntries* random numbers.  We're going to treat them as separate datasets though and see if we can understand how the datasets compare to each other.



In [ ]:
#just generate the datasets
ntries = 200
ndatasets = 500
data = np.random.normal(0,1,size=(ntries,ndatasets))
#print(data)



Let's start by looking at a single dataset.  Maybe this is what Student A measured...

In [ ]:
nbins = 25
counts, bins, bars  = plt.hist(data[:,0],bins=nbins,histtype='step')


How many points are outside of $\pm 1 \sigma$?  How many are outside of $\pm 2 \sigma$.  Is that about what you expect based on the Gaussian distribution?

About how many datasets would you expect to need to see a count outside of $\pm 3 \sigma$?

In [ ]:
nbins = 25
for i in range(3):
  counts, bins, bars  = plt.hist(data[:,i],bins=nbins,histtype='step')


Now that we've generated the datasets we want to just plot them.  If ndatasets is pretty small, we can just overlay them.  We see that they are all different (as we expect) but that they all roughly look like our Gaussian that we expect.

The code below also puts the histogram bin content into a 2D array using *np.concatenate*.  This gives us an array which has *ntries* columns and *nbins* rows.

If we look near the center of the Gaussian we expect to have a lot more counts and we should be able to see that the bin contents themselves are Gaussian distributed around their mean.

In [ ]:
nbins = 25
counts, bins, bars  = plt.hist(data[:,0],bins=nbins,histtype='step')
bincontent_array = counts.reshape(-1,1)
guess=[1,1,1]
uncertainties = np.sqrt(counts)
bins = bins[:np.size(bins)-1]
print(np.size(counts),np.size(bins))

for i in range(ndatasets-1):
  counts, bins, bars  = plt.hist(data[:,i+1],bins=nbins,histtype='step')
  uncertainties = np.sqrt(counts)
  bins = bins[:np.size(bins)-1]
  bincontent_array = np.concatenate((counts.reshape(-1, 1), bincontent_array), axis=1)


In [ ]:
bincontent_array[12,:]
plt.hist(bincontent_array[10,:],bins=20)
plt.title("")
plt.legend(["dataset"])
plt.show()
print(bincontent_array[10,:].mean())
print(bincontent_array[10,:].std())

Not too bad for what we can generate in class and we expect this estimation to get better as we increase both *ntries* and *ndatasets*.  

**Least Squares Fitting**

Least squares fitting is implemented in scipy.  In order to use it, you define a function which is your functional form and fit parameters (here, *func*).

In [ ]:
def func(x, a, b, c):
    return a*np.exp(-0.5*(b-x)**2/(c**2))


In [ ]:
nbins = 20
counts, bins, bars  = plt.hist(data[:,0],bins=nbins)
guess=[100,0,1]
uncertainties = np.sqrt(counts)
bins = bins[:np.size(bins)-1]
print(np.size(counts),np.size(bins))
popt, pcov, infodict, mesg, ier = curve_fit(func, bins, counts,guess,uncertainties,full_output=True)
fitparameters = popt.reshape(-1,1)
for i in range(ndatasets-2):
  counts, bins, bars  = plt.hist(data[:,i+1],bins=nbins)
  uncertainties = np.sqrt(counts)
  bins = bins[:np.size(bins)-1]
  popt, pcov, infodict, mesg, ier = curve_fit(func, bins, counts,guess,uncertainties,full_output=True)
  #print(ier, mesg)
  if ier!=4: fitparameters = np.concatenate((popt.reshape(-1,1),fitparameters),axis=1)

#print(fitparameters)


If you are going to use the fit for something, you had better check to make sure it converges!  Sometimes it does not!

we're going to use: https://lmfit.github.io/lmfit-py/model.html scipy.optimize doesn't cut it here because there isn't a way to access the fit quality information without hand-coding the $\chi^2$ calculation (happy to be shown to have just missed it).

In [ ]:
from numpy import exp, loadtxt, pi, sqrt
!pip install lmfit
from lmfit import Model


x = bins
y = counts


def gaussian(x, amp, cen, wid):
    """1-d gaussian: gaussian(x, amp, cen, wid)"""
    return (amp / (sqrt(2*pi) * wid)) * exp(-(x-cen)**2 / (2*wid**2))


gmodel = Model(gaussian)
result = gmodel.fit(y, x=x, weights=1.0/uncertainties, amp=5, cen=5, wid=1)

print(result.fit_report())

plt.plot(x, y, 'o')
plt.plot(x, result.init_fit, '--', label='initial fit')
plt.plot(x, result.best_fit, '-', label='best fit')
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(1,3)
fig.suptitle('fit results, amplitude, mean, and sigma')
axs[0].hist(fitparameters[0,:],bins=20)
axs[1].hist(fitparameters[1,:],bins=20)
axs[2].hist(fitparameters[2,:],bins=20)



